In [1]:
import findspark

findspark.init('/opt/spark')

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [3]:
spark = SparkSession.builder.appName('lec_23_moynihan_ex2').getOrCreate()

22/04/22 16:41:59 WARN Utils: Your hostname, GPUServer resolves to a loopback address: 127.0.1.1; using 10.4.10.8 instead (on interface enp2s0)
22/04/22 16:41:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/04/22 16:42:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
dataScheme = StructType().add('tweet_id', 'long').add('tweet','string').add('user_id','integer')

In [9]:
streaming = (
    spark.readStream.schema(dataScheme).csv('hdfs://localhost:9000/ds420_shared/lec23')
)

In [10]:
streaming.isStreaming

True

In [11]:
streaming.printSchema()

root
 |-- tweet_id: long (nullable = true)
 |-- tweet: string (nullable = true)
 |-- user_id: integer (nullable = true)



In [12]:
user_count = streaming.groupBy('user_id').count().orderBy(F.desc('count'))

In [13]:
# Query

activityQuery = (
    user_count.writeStream.
    queryName('user_count').
    format('memory').
    outputMode('complete').
    start()
    
)

22/04/22 16:42:53 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-1790c024-c084-43b1-99ed-13268f9edfbb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


In [14]:
import time 

for x in range(10):
    df = spark.sql('SELECT * FROM user_count')
    if df.count()>0:
        df.show(10)
    time.sleep(5)

+-------+-----+
|user_id|count|
+-------+-----+
|   null| 1156|
|     12|    4|
|     13|    4|
|     16|    4|
|     19|    4|
|     15|    4|
|     17|    4|
|     10|    4|
|     11|    4|
|     14|    4|
+-------+-----+
only showing top 10 rows

+-------+-----+
|user_id|count|
+-------+-----+
|   null| 1156|
|     12|    4|
|     13|    4|
|     16|    4|
|     19|    4|
|     15|    4|
|     17|    4|
|     10|    4|
|     11|    4|
|     14|    4|
+-------+-----+
only showing top 10 rows

+-------+-----+
|user_id|count|
+-------+-----+
|   null| 1156|
|     12|    4|
|     13|    4|
|     16|    4|
|     19|    4|
|     15|    4|
|     17|    4|
|     10|    4|
|     11|    4|
|     14|    4|
+-------+-----+
only showing top 10 rows

+-------+-----+
|user_id|count|
+-------+-----+
|   null| 1156|
|     12|    4|
|     13|    4|
|     16|    4|
|     19|    4|
|     15|    4|
|     17|    4|
|     10|    4|
|     11|    4|
|     14|    4|
+-------+-----+
only showing top 10 rows



In [15]:
file.stop()

NameError: name 'file' is not defined

In [7]:
activityQuery.stop()

NameError: name 'activityQuery' is not defined